In [1]:
import classiq
classiq.authenticate()

/opt/anaconda3/envs/baler/lib/python3.11/site-packages/classiq/_internals/authentication/token_manager.py:101: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from typing import List
import pdb

from classiq import (
    synthesize,
    qfunc,
    QArray,
    QBit,
    RX,
    CArray,
    Output,
    CReal,
    allocate,
    repeat,
    create_model,
    show
)

# choosing our data
import torchvision
import torchvision.transforms as transforms
import math
from classiq.applications.qnn.types import SavedResult
from classiq.applications.qnn import QLayer
from classiq.qmod.symbolic import pi
from classiq.synthesis import SerializedQuantumProgram

/opt/anaconda3/envs/baler/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/baler/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/anaconda3/envs/baler/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <CFB4297E-118A-3D27-BF39-55106327812A> /opt/anaconda3/envs/baler/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
from classiq.execution import (
    ExecutionPreferences,
    execute_qnn,
    set_quantum_program_execution_preferences,
)
from classiq.synthesis import SerializedQuantumProgram
from classiq.applications.qnn.types import (
    MultipleArguments,
    ResultsCollection,
    SavedResult,
)


from classiq.applications.qnn.types import (
    MultipleArguments,
    ResultsCollection,
    SavedResult,
)

In [ ]:
N_QUBITS = 4
num_shots = 1024

In [5]:
def execute(
    quantum_program: SerializedQuantumProgram, arguments: MultipleArguments
) -> ResultsCollection:
    quantum_program = set_quantum_program_execution_preferences(
        quantum_program, preferences=ExecutionPreferences(num_shots=num_shots)
    )
    return execute_qnn(quantum_program, arguments)

In [6]:
def post_process(result: SavedResult) -> torch.Tensor:
    res = result.value
    yvec = [
        (res.counts_of_qubits(k)["1"] if "1" in res.counts_of_qubits(k) else 0)
        / num_shots
        for k in range(N_QUBITS)
    ]

    return torch.tensor(yvec)

In [7]:
def get_circuit():

    @qfunc
    def vqc(input_: CArray[CReal, N_QUBITS], weight_: CArray[CArray[CReal, N_QUBITS], N_QUBITS], res:  QArray[QBit]) -> None:
        num_qubits = input_.len
        num_qlayers = weight_.len
        repeat(
            count=num_qlayers,
            iteration=lambda i: repeat(count=num_qubits, 
                                           iteration=lambda j:RX(pi * weight_[i][j], res[j]))
        )
        
    @qfunc
    def angle_embedding(input_: CArray[CReal, N_QUBITS], res: QArray[QBit]) -> None:
        repeat(
            count=input_.len,
            iteration=lambda index: RX(pi * input_[index], res[index]),
        )

    @qfunc
    def main(input_: CArray[CReal, N_QUBITS], weight_: CArray[CArray[CReal, N_QUBITS], N_QUBITS], res: Output[QArray[QBit]]) -> None:
        allocate(N_QUBITS, res)
        angle_embedding(input_=input_, res=res)
        vqc(input_=input_, weight_=weight_, res=res)
        
    qmod = create_model(main)
    quantum_program  = synthesize(qmod)
    return quantum_program

In [8]:
class Patchify(torch.nn.Module):
    """
    Patchify layer implemented using the Conv2d layer
    """
    def __init__(self, patch_size:int, hidden_size:int):
        super(Patchify, self).__init__()
        self.patch_size = patch_size
        self.conv = torch.nn.Conv2d(in_channels=3, out_channels=hidden_size, kernel_size=self.patch_size, stride=self.patch_size)
        self.hidden_size = hidden_size
        
    def forward(self, x:torch.Tensor):
        bs, c, h, w = x.size()
        self.num_patches = (h // self.patch_size) ** 2

        x = self.conv(x)
        x = x.view(bs, self.num_patches, self.hidden_size)
        return x
    


In [9]:
class RotaryPositionalEmbedding(torch.nn.Module):
    """
    Rotary Positional Embedding
    """
    def __init__(self, d_model, max_seq_len):
        super(RotaryPositionalEmbedding, self).__init__()

        # Create a rotation matrix.
        self.rotation_matrix = torch.zeros(d_model, d_model)
        for i in range(d_model):
            for j in range(d_model):
                self.rotation_matrix[i, j] = math.cos(i * j * 0.01)

        # Create a positional embedding matrix.
        self.positional_embedding = torch.zeros(max_seq_len, d_model)
        for i in range(max_seq_len):
            for j in range(d_model):
                self.positional_embedding[i, j] = math.cos(i * j * 0.01)

    def forward(self, x):
        """
        Args:
            x: A tensor of shape (batch_size, seq_len, d_model).

        Returns:
            A tensor of shape (batch_size, seq_len, d_model).
        """

        # Add the positional embedding to the input tensor.
        x += self.positional_embedding

        # Apply the rotation matrix to the input tensor.
        x = torch.matmul(x, self.rotation_matrix)

        return x

In [10]:
class QuantumLayer(torch.nn.Module):
    """
    Quantum Layer
    """
    def __init__(self, in_dim, out_dim):
        super(QuantumLayer, self).__init__()
        self.quantum_program = get_circuit()
        self.quantum_layer = QLayer(self.quantum_program, execute, post_process)

    def forward(self, x:torch.Tensor):
        size = x.size()
        x = x.view(-1, size[-1])
        x = self.quantum_layer(x)
        x = x.view(size)
        return x

In [11]:
class FFN(torch.nn.Module):
    """
    Feed Forward Network
    """
    def __init__(self, in_dim, hidden_size):
        super().__init__()
        self.linear_1 = torch.nn.Linear(in_dim, hidden_size)
        self.qlinear_1 = QuantumLayer(hidden_size, hidden_size)
        self.dropout = torch.nn.Dropout(p=0.4)
        self.linear_2 = torch.nn.Linear(hidden_size, in_dim)
        return
    
    def forward(self, x:torch.Tensor):
        x = self.linear_1(x)
        x = self.qlinear_1(x)
        x = self.dropout(x)
        x = torch.nn.functional.gelu(x)
        x = self.linear_2(x)
        return x

In [12]:
class qMHA(torch.nn.Module):
    """
    Quantum Multihead Attention
    """
    def __init__(self, in_dim:int, num_heads:int) -> None:
        super().__init__()

        self.qK = QuantumLayer(in_dim, in_dim);
        self.qQ = QuantumLayer(in_dim, in_dim);
        self.qV = QuantumLayer(in_dim, in_dim);
        self.dropout = torch.nn.Dropout(p=0.1)
        
        self.final_l = QuantumLayer(in_dim, in_dim)
        self.num_heads = num_heads
        self.in_dim = in_dim
        
        return

    def forward(self, X:torch.Tensor):

        dim = torch.sqrt(torch.Tensor([X.shape[-1]]))
        attention = torch.nn.functional.softmax((1/dim)*self.qK(X))*self.qQ(X)*self.qV(X)
        x = self.dropout(attention)
        x = self.final_l(x)
        return x

In [13]:
class qTransformerEncoder(torch.nn.Module):
    """
    Quantum Transformer Encoder Layer
    """
    def __init__(self, in_dim:int, num_heads:int) -> None:
        super().__init__()
        self.layer_norm_1 = torch.nn.LayerNorm(normalized_shape=in_dim)
        self.layer_norm_2 = torch.nn.LayerNorm(normalized_shape=in_dim)
        
        self.qMHA = qMHA(in_dim, num_heads)
        self.qFFN = FFN(in_dim, hidden_size=in_dim)
        self.dropout = torch.nn.Dropout(p=0.3)
        

    def forward(self, X:torch.Tensor):
        
        x = self.qMHA(X)
        x = self.layer_norm_1(x)
        x = self.dropout(x) + X

        y = self.layer_norm_2(x)
        y = self.qFFN(y)+y
        
        return x

In [20]:
class QVT(torch.nn.Module):
    """
    Quantum Vision Transformer;
    """
    def __init__(self, patch_size, in_dim, hidden_size,  num_heads, n_classes, n_layers) -> None:
        super().__init__()
        self.patch_formation = Patchify(patch_size=patch_size, hidden_size=hidden_size)
        self.d_model = (in_dim//patch_size)**2
        self.pos_encoding = RotaryPositionalEmbedding(hidden_size, self.d_model)
        self.transformer_blocks = [qTransformerEncoder(hidden_size, num_heads) for i in range(n_layers)]
        
        self.n_classes = n_classes
        self.final_normalization = torch.nn.LayerNorm(hidden_size)
        self.final_layer = torch.nn.Linear(hidden_size, self.n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:        
        x = self.patch_formation(x)
        x += self.pos_encoding(x)
        for trans_block in self.transformer_blocks:
            x = trans_block(x)
        x = self.final_normalization(x)
        x = x.mean(axis=1)
        x = self.final_layer(x)
        return x

In [21]:
x = torch.rand(32, 3, 16, 16)
qVisTransformer = QVT(patch_size=4, in_dim=16, hidden_size=4, num_heads=1, n_classes=10, n_layers=4)
res = qVisTransformer(x)

/var/folders/zj/jmm9ywc51bb4p1pn12_23y340000gn/T/ipykernel_4329/535465168.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention = torch.nn.functional.softmax((1/dim)*self.qK(X))*self.qQ(X)*self.qV(X)


In [22]:
res.size()

torch.Size([32, 10])